In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

#Dimensions of RR tape
REAL_HEIGHT = 5 #cm
REAL_WIDTH = 13 #cm

#RR tape masks
hsvLow = np.array([45, 200, 200])
hsvHigh = np.array([75, 255, 255])

#Limelight constants
PIXELS_WIDE = 320 #pixels
PIXELS_HIGH = 240 #pixels
FOV_Y = 49.7 #degrees
FOV_X = 59.6 #degrees
FOCAL_LENGTH = (PIXELS_HIGH / 2) / np.tan(math.pi / 180 * FOV_Y / 2)

#Other parameters
TARGET_HEIGHT = 240 #cm (measured from the top of the target to the ground)
CAMERA_HEIGHT = 114.3 #cm
CAMERA_ANGLE = 20 # degrees

contour functions

In [ ]:
#returns a contour that follows the following criteria: largest contour thats aspect ratio is between 2 and 3
def contour_criteria(contours):
  max = 0
  for i in range(1, len(contours)):
    x, y, width, height = cv2.boundingRect(contours[i])
    if cv2.contourArea(contours[i]) > cv2.contourArea(contours[max]) and width / height > 2 and width / height < 3:
        max = i
  return contours[max]
  
#returns the contour of one of the detected RR tapes
def getGoalContour(image):
  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  mask = cv2.inRange(hsv, hsvLow, hsvHigh)
  contours, hiearchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  #If there are no contours found in the image, None is returned
  if(len(contours) > 0):
    contour = contour_criteria(contours)
    return contour
  return None

Distance & turn angle things

In [ ]:
#returns the horizontal distance (cm) from the RR tape to the camera
def getDistance(contour):
  x, y, width, height = cv2.boundingRect(contour)
  
  #normalizing the top y position of the goal contour to be from [1,-1]
  norm_Y = - (y - (PIXELS_HIGH / 2)) / (PIXELS_HIGH / 2)

  #finding the vertical angle (pitch) between the camera and the goal contour 
  pitch = 180 * np.arctan(norm_Y * np.tan(FOV_Y / 2 * math.pi / 180)) / math.pi # degrees

  #using trig to determine distance
  distance = (TARGET_HEIGHT - CAMERA_HEIGHT) / np.tan(math.pi * (CAMERA_ANGLE + pitch) / 180)

  return distance

#returns angle (deg) of the RR tape relative to the camera
def getAngle(contour):
  x, y, width, height = cv2.boundingRect(contour)

  #normalizing the left x position of the goal contour to be from [1,-1]
  norm_X = (x - (PIXELS_WIDE / 2)) / (PIXELS_WIDE / 2)

  #finding the horizontal angle between the camera and left edge of the goal contour
  angle = 180 * np.arctan(norm_X * np.tan(FOV_X / 2 * math.pi / 180)) / math.pi #degrees

  return angle

#TESTING
REAL_DIST = 10 #ft
img_test = cv2.imread(str(REAL_DIST) + 'ft.jpg')
contours, hiearchy = cv2.findContours(cv2.inRange(cv2.cvtColor(img_test, cv2.COLOR_BGR2HSV), np.array([0, 70, 100]), np.array([20, 255, 255])), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour = contour_criteria(contours)
plt.imshow(cv2.drawContours(cv2.cvtColor(img_test.copy(), cv2.COLOR_BGR2RGB), [contour], -1, (0, 255, 0), 3), cmap='gray')
plt.show()
distance = getDistance(contour)
angle = getAngle(contour)
print('Calculated Distance: ' + str(distance))
print('Real Distance: ' + str(REAL_DIST * 30))
print('Turn Angle: ' + str(angle))


Best pipeline ever to step foot in a code cell

In [ ]:
def runPipeline(image, llrobot):
  #do stuff
  goal = getGoalContour(image)
  distance = getDistance(goal)
  angle = getAngle(goal)
  output = [distance, angle]

  #DONT TOUCH
  low_threshold = np.array([240, 0, 0])
  high_threshold = np.array([255, 255, 255])
  binary = np.all((image > low_threshold) * (image <= high_threshold), axis=2)
  mask = np.where(binary, 255, 0)
  output_image = image.copy()
  output_image[mask > 0] = [255, 255, 255]
  return ([[[0, 0]], [[0, 100]], [[100, 0]], [[100, 100]]], output_image, output)